### What is ARIMA:
ARIMA stands for Autoregressive Integrated Moving Average. It is an algorithm used for forecasting Time Series Data. ARIMA models have three parameters like ARIMA(p, d, q).  

1. **p** is the number of lagged values that need to be added or subtracted from the values (label column). It captures the autoregressive part of ARIMA.
2. **d** represents the number of times the data needs to differentiate to produce a stationary signal. If it’s stationary data, the value of d should be 0, and if it’s seasonal data, the value of d should be 1. d captures the integrated part of ARIMA.
3. **q** is the number of lagged values for the error term added or subtracted from the values (label column). It captures the moving average part of ARIMA.

In [2]:
import pandas as pd
import yfinance as yf
import datetime
from datetime import date, timedelta

In [4]:
today = date.today()
d1 = today.strftime("%Y-%m-%d")
end_date = d1
d2 = today - timedelta(days=365)
start_date = d2

print("Start Date", start_date, "End Date", end_date)

Start Date 2023-12-25 End Date 2024-12-24


In [5]:
# Collecting Google Stock Price using Yahoo Finance API
dataset = yf.download('GOOG',
                      start=start_date,
                      end=end_date, 
                      progress=False)

In [6]:
dataset.head()

Price,Adj Close,Close,High,Low,Open,Volume
Ticker,GOOG,GOOG,GOOG,GOOG,GOOG,GOOG
Date,,,,,,
2023-12-26,142.308670,142.820007,143.945007,142.500000,142.979996,11170100
2023-12-27,140.933624,141.440002,143.320007,141.050995,142.830002,17288400
2023-12-28,140.774185,141.279999,142.270004,140.828003,141.850006,12192500
2023-12-29,140.425446,140.929993,141.434998,139.899994,140.679993,14872700
2024-01-02,139.060333,139.559998,140.615005,137.740005,139.600006,20071900
